In [1]:
import lxw
import pandas as pd
import re

# 1. 24点问题

需要对Thought Decomposition、Thought Generator、State Evaluator进行精心的示例设计，泛化性低。

In [2]:
# 5-shot
standard_prompt = '''Use numbers and basic arithmetic operations (+ - * /) to obtain 24.
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) = 24
Input: 2 9 10 12
Answer: 2 * 12 * (10 - 9) = 24
Input: 4 9 10 13
Answer: (13 - 9) * (10 - 4) = 24
Input: 1 4 8 8
Answer: (8 / 4 + 1) * 8 = 24
Input: 5 5 5 9
Answer: 5 + 5 + 5 + 9 = 24
Input: {input}
'''

# 5-shot
cot_prompt = '''Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Each step, you are only allowed to choose two of the remaining numbers to obtain a new number.
Input: 4 4 6 8
Steps:
4 + 8 = 12 (left: 4 6 12)
6 - 4 = 2 (left: 2 12)
2 * 12 = 24 (left: 24)
Answer: (6 - 4) * (4 + 8) = 24
Input: 2 9 10 12
Steps:
12 * 2 = 24 (left: 9 10 24)
10 - 9 = 1 (left: 1 24)
24 * 1 = 24 (left: 24)
Answer: (12 * 2) * (10 - 9) = 24
Input: 4 9 10 13
Steps:
13 - 10 = 3 (left: 3 4 9)
9 - 3 = 6 (left: 4 6)
4 * 6 = 24 (left: 24)
Answer: 4 * (9 - (13 - 10)) = 24
Input: 1 4 8 8
Steps:
8 / 4 = 2 (left: 1 2 8)
1 + 2 = 3 (left: 3 8)
3 * 8 = 24 (left: 24)
Answer: (1 + 8 / 4) * 8 = 24
Input: 5 5 5 9
Steps:
5 + 5 = 10 (left: 5 9 10)
10 + 5 = 15 (left: 9 15)
15 + 9 = 24 (left: 24)
Answer: ((5 + 5) + 5) + 9 = 24
Input: {input}
'''

# 1-shot
propose_prompt = '''Input: 2 8 8 14
Possible next steps:
2 + 8 = 10 (left: 8 10 14)
8 / 2 = 4 (left: 4 8 14)
14 + 2 = 16 (left: 8 8 16)
2 * 8 = 16 (left: 8 14 16)
8 - 2 = 6 (left: 6 8 14)
14 - 8 = 6 (left: 2 6 8)
14 /  2 = 7 (left: 7 8 8)
14 - 2 = 12 (left: 8 8 12)
Input: {input}
Possible next steps:
'''

value_prompt = '''Evaluate if given numbers can reach 24 (sure/likely/impossible)
10 14
10 + 14 = 24
sure
11 12
11 + 12 = 23
12 - 11 = 1
11 * 12 = 132
11 / 12 = 0.91
impossible
4 4 10
4 + 4 + 10 = 8 + 10 = 18
4 * 10 - 4 = 40 - 4 = 36
(10 - 4) * 4 = 6 * 4 = 24
sure
4 9 11
9 + 11 + 4 = 20 + 4 = 24
sure
5 7 8
5 + 7 + 8 = 12 + 8 = 20
(8 - 5) * 7 = 3 * 7 = 21
I cannot obtain 24 now, but numbers are within a reasonable range
likely
5 6 6
5 + 6 + 6 = 17
(6 - 5) * 6 = 1 * 6 = 6
I cannot obtain 24 now, but numbers are within a reasonable range
likely
10 10 11
10 + 10 + 11 = 31
(11 - 10) * 10 = 10
10 10 10 are all too big
impossible
1 3 3
1 * 3 * 3 = 9
(1 + 3) * 3 = 12
1 3 3 are all too small
impossible
{input}
'''

value_last_step_prompt = '''Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Given an input and an answer, give a judgement (sure/impossible) if the answer is correct, i.e. it uses each input exactly once and no other numbers, and reach 24.
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) = 24
Judge: 
sure
Input: 2 9 10 12
Answer: 2 * 12 * (10 - 9) = 24
Judge: 
sure
Input: 4 9 10 13
Answer: (13 - 9) * (10 - 4) = 24
Judge: 
sure
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) + 1 = 25
Judge: 
impossible
Input: 2 9 10 12
Answer: 2 * (12 - 10) = 24
Judge: 
impossible
Input: 4 9 10 13
Answer: (13 - 4) * (10 - 9) = 24
Judge: 
impossible
Input: {input}
Answer: {answer}
Judge:'''

# 2. ToT Prompting

## 2.1 Prompts

---

Imagine three different experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realises they're wrong at any point then they leave. The question is: <br>
想象有三位不同的专家在回答这个问题。每位专家会写下他们思考的第一个步骤，然后与小组分享。接着，所有专家会继续进行下一个步骤，以此类推。只要大家发现有专家的步骤出错了，就让这位专家离开。问题是：


---

Simulate three brilliant, logical experts collaboratively answering a question. Each one verbosely explains their thought process in real-time, considering the prior explanations of others and openly acknowledging mistakes. At each step, whenever possible, each expert refines and builds upon the thoughts of others, acknowledging their contributions. They continue until there is a definitive answer to the question. For clarity, your entire response should be in a markdown table. The question is... <br>
模拟三位聪明且逻辑严谨的专家共同回答一个问题。每位专家都会详细解释他们的思考过程，考虑到他人之前的解释，并公开承认错误。在每一步中，每位专家都会在可能的情况下改进并基于他人的思路，认可他们的贡献。他们会一直继续，直到得出明确的答案。为了清晰起见，你的整个回答应以 Markdown 表格的形式呈现。问题是：

---

Identify and behave as three different experts that are appropriate to answering this question.
All experts will write down the step and their thinking about the step, then share it with the group.
Then, all experts will go on to the next step, etc.
At each step all experts will score their peers response between 1 and 5, 1 meaning it is highly unlikely, and 5 meaning it is highly likely.
If any expert is judged to be wrong at any point then they leave.
After all experts have provided their analysis, you then analyze all 3 analyses and provide either the consensus solution or your best guess solution.
The question is... <br>
确定并扮演三个适合回答此问题的专家。每位专家将写下步骤和他们对该步骤的思考，然后与小组分享。接着，所有专家会继续进行下一个步骤，以此类推。在每一步中，所有专家会对他们同伴的回答进行评分，评分范围为1到5分，1分表示非常不可能，5分表示非常可能。如果任何专家在某一时刻被认为是错误的，他们将退出。在所有专家提供他们的分析后，你需要分析这三份分析，并提供一致的解决方案或你认为最好的解决方案。问题是：

In [4]:
def Direct_IO(question, model):
    messages = [
        {
            "role": "user",
            "content": question
        }
    ]

    if not model:
        model = lxw.gpt4_turbo_1106

    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

In [28]:
def ToT_Prompt_1(question, model = None):
    prompt = (
        'Imagine three different experts are answering this question. \r\n'
        'All experts will write down 1 step of their thinking, then share it with the group. \r\n'
        'Then all experts will go on to the next step, etc. \r\n'
        'If any expert realises they are wrong at any point then they leave. \r\n'
        'The question is: \r\n'
    )
        
    messages = [
        {
            "role": "user",
            "content": prompt + '```\r\n' + question + '\r\n```'
        }
    ]

    if not model:
        model = lxw.gpt4_turbo_1106

    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        max_tokens = 4000,
    )
    return response.choices[0].message.content

In [38]:
def ToT_Prompt_1_CN(question, model = None):
    prompt = (
        '想象由三位不同的专家来共同回答这个问题的完整过程。\r\n'
        '所有专家都写下他们思考这个问题的第1个步骤，然后与大家分享。\r\n'
        '接下来，所有专家都写下他们思考的下一个步骤并分享。\r\n'
        '以此类推，直到所有专家写完他们思考的所有步骤。\r\n'
        '只要发现有专家的步骤出错了，就让这位专家离开。\r\n'
        '问题是: \r\n'
    )
        
    messages = [
        {
            "role": "user",
            "content": prompt + '```\r\n' + question + '\r\n```'
        }
    ]

    if not model:
        model = lxw.gpt4_turbo_1106

    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

In [45]:
def ToT_Prompt_1_Final(question, model = None):
    system_prompt = '''
# Profile
You simulate three brilliant logic experts collaborating to answer questions. The three experts are Expert1, Expert2, and Expert3

# Workflow
1. User input the question.
2. All experts will write down 1 step of their thinking, then share it with the group.
3. Then all experts will go on to the next step, etc.
4. If any expert realises they are wrong at any point then they leave.
5. Repeat until a final conclusion is reached.

# Constraint
Your final conclusion section should be in Chinese, while the rest of the content should be in English.

# Output Format
-- Step 1 --
Expert1: << thinking of expert1 in step 1 >>
Expert2: << thinking of expert2 in step 1 >>
Expert3: << thinking of expert3 in step 1 >>
-- Step 2 --
Expert1: << thinking of expert1 in step 2 >>
Expert2: << thinking of expert2 in step 2 >>
Expert3: << thinking of expert3 in step 2 >>
-- Step 3 --
...
...
...
-- Final Conclusion --
<< the final conclusion >>
'''
        
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            "role": "user",
            "content": question
        }
    ]

    if not model:
        model = lxw.gpt4_turbo_1106

    client = lxw.new_client(model = model)

    response = client.chat.completions.create(
        model = model,
        messages = messages
    )
    return response.choices[0].message.content

## 2.2 小望与鸡蛋问题

用直接提示，偶尔正确偶尔错误，有较高的错误率。

In [24]:
q = (
    '小望坐在客厅里。'
    '他站起来，手里拿了一个杯子走到厨房。'
    '他把一个鸡蛋放进杯子里，然后把杯子带到卧室。'
    '他将杯子倒置，然后走到花园。'
    '他把杯子放在花园里，然后走到车库。'
    '他开上车，去找波波约会。'
    '鸡蛋在哪里？'
)
print(Direct_IO(q, model = lxw.gpt4_turbo_1106))

鸡蛋最初被小望放进了杯子里。即使后来他走到不同的地方，除非有特别的提及他取出鸡蛋或杯子发生了什么，我们可以假设鸡蛋始终在杯子里。因此，当小望将杯子倒置放在花园里后，鸡蛋应该仍然在杯子里。所以，鸡蛋现在在花园里，倒置的杯子里面。


In [26]:
print(ToT_Prompt_1(q, model = lxw.gpt4_turbo_1106))

Expert 1:
Step 1: Translate and understand the story. 小望 (Xiaowang) is in the living room, he stands up with a cup in his hand and walks to the kitchen. He puts an egg into the cup and then takes the cup to the bedroom. He turns the cup upside down and then walks to the garden, where he puts the cup. Finally, he walks to the garage, gets into his car, and leaves for a date with 波波 (Bobo). My understanding is that the egg is now somewhere between the bedroom and the garden.

Expert 2:
Step 1: First, I need to clarify the sequence of events and understand the logic of the story. Based on the given information, Xiaowang's actions seem to suggest that he has transported the egg from the kitchen to the bedroom in the cup. The most critical part to focus on is what happened to the egg after he turned the cup upside down in the bedroom, before going to the garden.

Expert 3:
Step 1: Identify the last known location of the egg. The last explicit mention of the egg's location is when Xiaowang p

In [39]:
print(ToT_Prompt_1_CN(q, model = lxw.gpt4_turbo_1106))

专家A（逻辑专家）：
第1步：理解并分析问题，整理已知情况。小望有一个杯子，他将鸡蛋放进杯子里，并且把带有鸡蛋的杯子带到了卧室。

专家B（物理专家）：
第1步：根据物理定律，考虑涉及问题中的对象的物理属性。杯子是一个容器，能够容纳鸡蛋，鸡蛋是一个脆弱的物体，放在杯子里时会随着杯子一起移动。

专家C（心理学专家）：
第1步：考虑小望的行为动机和心理状态。小望为什么要把鸡蛋从厨房带到卧室，这个行为背后可能有什么特殊的意图？是否有什么情感上的依恋使他将鸡蛋带在身边？

---

专家A（逻辑专家）：
第2步：继续沿着小望的行为链条推理。小望将杯子颠倒，然后前往花园。需要考虑他颠倒杯子的行为对鸡蛋的位置的影响。

专家B（物理专家）：
第2步：根据物理定律分析小望颠倒杯子的行为可能产生的结果。颠倒一个装有鸡蛋的杯子会使鸡蛋落出。

专家C（心理学专家）：
第2步：分析小望将杯子倒置的潜在心理因素。可能是一种仪式，或者是试图在自己的私密空间里营造一个特别的记忆。

---

专家A（逻辑专家）：
第3步：结合所有已知的行动序列分析出鸡蛋所在的位置。小望颠倒杯子的行为意味着杯子内的鸡蛋会掉出。因此，鸡蛋应该在小望倒置杯子时的地点 — 卧室。

专家B（物理专家）：
第3步：进一步探讨在倒置杯子过程中对鸡蛋状态的影响。倒置杯子一般情况下意味着其中的物品会因重力作用而掉落。所以鸡蛋应该不在杯子里，它应该掉在了卧室的地上。

专家C（心理学专家）：
第3步：从心理学角度推断鸡蛋的位置。由于小望执行了一连串有意义的行为，可能出于某种心理因素，他选择在特定的地方留下鸡蛋。鸡蛋最可能仍在他倒置杯子的房间内，也就是卧室。

---

所有专家A、B、C的步骤一致表明鸡蛋在卧室。无需让任何一位专家离开。


In [46]:
print(ToT_Prompt_1_Final(q, model = lxw.gpt4_turbo_1106))

-- Step 1 --
Expert1: The scenario describes a sequence of actions involving Xiaowang, a cup, and an egg. I will follow the sequence to determine the final location of the egg.
Expert2: I agree with Expert1's approach. Tracking the movements of Xiaowang with the egg in the cup should allow us to establish where the egg is.
Expert3: There may be a trick in the sequence of actions described. The egg's final location will depend on Xiaowang's actions with the cup.

-- Step 2 --
Expert1: At first, Xiaowang puts the egg in the cup in the kitchen, and then he carries the cup to the bedroom. This means that the egg was inside the cup when he moved to the bedroom.
Expert2: The statement says Xiaowang inverts the cup in the bedroom. If there is no further interaction with the egg, the inversion suggests that the egg is no longer contained within the cup.
Expert3: If we presume that the cup was inverted while the egg was inside it, the egg would not be in the cup anymore unless it was somehow se

通益千问与GPT对比测试：<br>

想象由三位不同的专家来共同回答这个问题的完整过程。 \
所有专家都写下他们思考这个问题的第1个步骤，然后与大家分享。你输出第1个步骤的所有专家的结果以及讨论意见。 \
接下来，所有专家都写下他们思考的下一个步骤并分享。你输出这个步骤的所有专家的结果以及讨论意见。 \
以此类推，直到所有专家写完他们思考的所有步骤。 \
只要发现有专家的步骤出错了，就让这位专家离开。 \
问题是：
```
小望坐在客厅里。
他站起来，手里拿了一个杯子走到厨房。
他把一个鸡蛋放进杯子里，然后把杯子带到卧室。
他将杯子倒置，然后走到花园。
他把杯子放在花园里，然后走到车库。
他开上车，去找波波约会。
鸡蛋在哪里？
```